In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import os
import os.path as path

from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [ ]:
sns.set(style="whitegrid")
plt.rcParams.update({'figure.figsize': (7, 5), 'figure.dpi': 120})

# Exploratory data analysis

In [ ]:
df = pd.read_csv('../data/raw/stoloto_data.csv',
                       sep=',',
                       parse_dates=['date'], index_col=0)

### A glance at the dataset

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
# So we have 30021 values for all columns, no NaN values.
# I don't see much physical meaning in statistics for categolical features such game_code, postamt_num, ufps_num
# Hmm, std for 'ufps_num' is 0. Let's check is it a constant
df['ufps_num'].value_counts() 

In [ ]:
# Yes, it is. It's useless, let's delete it
del df['ufps_num']

In [ ]:
game_codes_counts = df['game_code'].value_counts()
# game_code have only 5 possible values. Probably we should change this feature into 5 dummy variables in the regression
game_codes = game_codes_counts.index
print(game_codes)

In [ ]:
df['postamt_num'].value_counts()
# Again, postamt_num consist of 6 groups about the same size and can be replaced with 6 dummy variables.

In [ ]:
# Let's take a look at spreading sales by combination of postamt_num and game_code
ops_num_game_code_sales = df.pivot_table(
                        index='postamt_num', 
                        columns='game_code', 
                        values='sales', 
                        aggfunc=sum).fillna(0).applymap(float)
sns.heatmap(ops_num_game_code_sales, annot=True, fmt=".1f", linewidths=.5)

In [ ]:
df['circulation'].nunique()

In [ ]:
df['ops_num'].nunique()

So in theory we have 5 game_codes * 93 ops_nums = 465 time series to make s forecast

#### Features correlation check

In [ ]:
df.corr()

In [ ]:
plt.matshow(df.corr())
plt.show()
#We have weak correlation between circulation and game_code (-0.35) but we still can work with it.

#### Target variable survey

In [ ]:
common_sales = df.sort_values(by=['date']).groupby('date').sum()['sales']
common_sales.plot(figsize=(8,6))
plt.ylabel('sales')
plt.title('Sales by weeks')
plt.show()

We can see two big outlaers in the end of two last years

#### Prepare Time Series

In [ ]:
df_grouped = df.sort_values(by=['date']).groupby(['game_code', 'ops_num', 'date'])['sales'].sum()
multiindex_count = df_grouped.index.to_frame()
temp_df = pd.DataFrame(multiindex_count[['game_code', 'ops_num', 'date']].values, columns=['game_code', 'ops_num', 'date'])
temp_df['sales'] = df_grouped.values
multi_ts = df_grouped.unstack([0, 1])
multi_ts.head(10)

In [ ]:
multi_ts.isna().sum()

In [ ]:
ts_lengths = pd.DataFrame(len(multi_ts.index)- multi_ts.isna().sum(), columns=['ts_length']).sort_values(by=['ts_length'])

plt.hist(ts_lengths['ts_length'])
plt.xlabel('Time series')
plt.ylabel('Number of observations')
plt.title('Histogram of TS lengths')
plt.figure(figsize=(4,2.5))
plt.show()

In [ ]:
# We have 436 time series most of which consist of 22-50 values
# We need to deside the minimum length of time series to predict. Let's assume it to 30
max_number_of_nas = len(multi_ts.index) - 30
ts_min30length = multi_ts.loc[:, (multi_ts.isna().sum(axis=0) <= max_number_of_nas)]
ts_df = pd.DataFrame() 
for i, col in enumerate(ts_min30length.columns):
    ts_df.insert(i, str(col[0])+'_'+str(col[1]), ts_min30length.iloc[:, i])
ts_df.shape

In [ ]:
ts_df.head()

We left 350 time series for individual forecasting with minimum 30 values

Let's take a look at 10 random time series

In [ ]:
ts_rand = ts_df.sample(10, axis=1)
ts_rand.plot(subplots=True, layout=(5,2), figsize=(8,15))
plt.ylabel('sales')
plt.title('10 random TS of sales by game_code and ops_num')
plt.show()

## Fill missing values

There are lot of NaN values in our time series. We have several approaches of imputation.

In [ ]:
random2ts = ts_df.sample(2, axis=1, random_state=1)
random2ts.plot(subplots=True, layout=(1,2), figsize=(8,4))

In [ ]:
# Fill in with 0.
# Since there are sales by certain game type in every post office for each week so we can assume that no value means 0 sales.
# But if it is not true and data is't complete probably we shouid choose more complicatetd approach such as Backward Fill, Linear Interpolation, Quadratic interpolation, Mean of nearest neighbors
df_zeros = ts_df.fillna(0)
df_zeros[random2ts.columns].plot(subplots=True, layout=(1,2), figsize=(8,4))

In [ ]:
## 2. Forward Fill
df_ffill = ts_df.ffill().fillna(0)
fig, axes = plt.subplots(1, 2, figsize=(10,5))
df_ffill[random2ts.columns[0]].plot(subplots=True, title='Forward Fill', ax=axes[0], label='Forward Fill', color='red', style=".-")
ts_df[random2ts.columns[0]].plot(subplots=True, title='Forward Fill', ax=axes[0], label='Forward Fill', color='green', style=".-")
axes[0].legend(["Forward Filled Data", "Available Data"])
df_ffill[random2ts.columns[1]].plot(subplots=True, title='Forward Fill', ax=axes[1], label='Forward Fill', color='red', style=".-")
ts_df[random2ts.columns[1]].plot(subplots=True, title='Forward Fill', ax=axes[1], label='Forward Fill', color='green', style=".-")
axes[1].legend(["Forward Filled Data", "Available Data"])

In [ ]:
## 3. Backward Fill 
df_bfill = ts_df.bfill()
fig, axes = plt.subplots(1, 2, figsize=(10,5))
df_bfill[random2ts.columns[0]].plot(subplots=True, title='Backward Fill ', ax=axes[0], label='Backward Fill', color='red', style=".-")
ts_df[random2ts.columns[0]].plot(subplots=True, title='Backward Fill', ax=axes[0], label='Backward Fill', color='green', style=".-")
axes[0].legend(["Backward Filled Data", "Available Data"])
df_bfill[random2ts.columns[1]].plot(subplots=True, title='Backward Fill', ax=axes[1], label='Backward Fill', color='red', style=".-")
ts_df[random2ts.columns[1]].plot(subplots=True, title='Backward Fill', ax=axes[1], label='Backward Fill', color='green', style=".-")
axes[1].legend(["Backward Filled Data", "Available Data"])

In [ ]:
# Output preprocessed data
__file__ = os.getcwd()
project_path = path.normpath(path.abspath(path.join(__file__,'../')))
df_bfill.to_csv(path.normpath(project_path +
                                '/data/preprocessed/bfilled_ts.csv'), index=True)

In [ ]:
## 4. Linear Interpolation
# df_linear = pd.DataFrame()
# ts_df['rownum'] = np.arange(ts_df.shape[0])
# for i, col in enumerate(ts_df.columns[:-1]):
#     df_nona = ts_df.dropna(subset = [col])
#     f = interp1d(df_nona['rownum'], df_nona[col])
#     df_linear[col] = f(ts_df[ts_df.index >= ts_df[col].notna().idxmax()]['rownum'])
# ts_df.drop(['rownum'], axis=1)

# fig, axes = plt.subplots(1, 2, figsize=(10,5))
# df_linear[random2ts.columns[0]].plot(subplots=True, title='Linear Interpolation', ax=axes[0], label='Linear Interpolation', color='red', style=".-")
# ts_df[random2ts.columns[0]].plot(subplots=True, title='Linear Interpolation', ax=axes[0], label='Linear Interpolation', color='green', style=".-")
# axes[0].legend(["Linear Interpolated Data", "Available Data"])
# df_linear[random2ts.columns[1]].plot(subplots=True, title='Linear Interpolation', ax=axes[1], label='Linear Interpolation', color='red', style=".-")
# ts_df[random2ts.columns[1]].plot(subplots=True, title='Linear Interpolation', ax=axes[1], label='Linear Interpolation', color='green', style=".-")
# axes[1].legend(["Linear Interpolated Data", "Available Data"])

In [ ]:
## 6. Mean of 'n' Nearest Past Neighbors
def knn_mean(ts, n):
    out = np.copy(ts)
    for i, val in enumerate(ts):
        if np.isnan(val):
            n_by_2 = np.ceil(n/2)
            lower = np.max([0, int(i-n_by_2)])
            upper = np.min([len(ts)+1, int(i+n_by_2)])
            ts_near = np.concatenate([ts[lower:i], ts[i:upper]])
            out[i] = np.nanmean(ts_near)
    return out

df_knn = ts_df.copy()
for col in df_knn.columns:
    while df_knn[col].isnull().sum() > 0:
        df_knn[col] = knn_mean(df_knn[col].values, 8)

fig, axes = plt.subplots(1, 2, figsize=(14,5))
df_knn[random2ts.columns[0]].plot(subplots=True, title='kNN', ax=axes[0], label='kNN', color='red', style=".-")
ts_df[random2ts.columns[0]].plot(subplots=True, title='kNN', ax=axes[0], label='kNN', color='green', style=".-")
axes[0].legend(["Mean of 'n' Nearest Past Neighbors", "Available Data"])
df_knn[random2ts.columns[1]].plot(subplots=True, title='kNN', ax=axes[1], label='kNN', color='red', style=".-")
ts_df[random2ts.columns[1]].plot(subplots=True, title='kNN', ax=axes[1], label='kNN', color='green', style=".-")
axes[1].legend(["Mean of 'n' Nearest Past Neighbors", "Available Data"])

In [ ]:
# Output preprocessed data
__file__ = os.getcwd()
project_path = path.normpath(path.abspath(path.join(__file__,'../')))
df_knn.to_csv(path.normpath(project_path +
                                '/data/preprocessed/knn_imputated_ts.csv'), index=True)

We can see that TS with different game_code may differ from each other. 

#### Game code TS

In [ ]:
game_code_df = temp_df.groupby('game_code').sum()
fig = plt.figure(figsize=(6, 4))
plt.ylabel('sales')
plt.title('Sales by game_code')
plt.plot(game_code_df, 'ro')

Sales value sugnificatly differ from game type

In [ ]:
game_code_ts = temp_df.groupby(['game_code', 'date']).sum()
ts_by_game_code = game_code_ts.unstack([0])
#ts_by_game_code.droplevel(0, axis=1)
ts_by_game_code.columns = [col[1] for col in ts_by_game_code.columns]
ts_by_game_code.plot()

TS have too strong outliers. Let's remove it to see other parts of plot.

In [ ]:
for i in range(5):
    print(ts_by_game_code.iloc[:, [i]].idxmax())

In [ ]:
import datetime
game_code_no_outlier = ts_by_game_code.drop([datetime.date(2017,12,31), datetime.date(2018,12,30)])

plt.ylabel('sales')
plt.title('Time Series of Sales by game_code')
plt.plot(game_code_no_outlier)

We can see similarity at time series trends. Let's investidate it by trend and seasonality

## Trend and Seasonality

In [ ]:
# Boxplot of Month-wise (Seasonal) and Year-wise (trend) Distribution
# Prepare data
#ts_by_game_code.rename(columns = {7101: '7101', 7103: '7103', 7105: '7105', 7115: '7115', 7175: '7175'}, inplace = True)
ts_by_years = ts_by_game_code.reset_index(inplace=False)

ts_by_years['year'] = [d.year for d in ts_by_years.date]
ts_by_years['month'] = [d.strftime('%b') for d in ts_by_years.date]
years = ts_by_years['year'].unique()
# Plot
for i, game_code in enumerate(game_codes):
    fig, axes = plt.subplots(1, 2, figsize=(10,10), dpi= 80)
    year_box = sns.boxplot(x='year', y=game_code, data=ts_by_years, ax=axes[0])
    month_box = sns.boxplot(x='month', y=game_code, data=ts_by_years)
    upper_whisker_limit = ts_by_years[game_code].quantile(0.75) + 1.6 * (ts_by_years[game_code].quantile(0.75) - ts_by_years[game_code].quantile(0.25))
    year_box.set(ylim=(1, upper_whisker_limit))
    month_box.set(ylim=(1,  upper_whisker_limit))
    # Set Title
    axes[0].set_title('Year-wise Box Plot\n(The Trend)\n' + str(game_code), fontsize=18); 
    axes[1].set_title('Month-wise Box Plot\n(The Seasonality)\n' + str(game_code), fontsize=18)
    plt.show()



For the first 3 time series (game types 7103, 7105 and 7115) we see strong uprising trend that testifies to stationarity.
We can see some seasonality - sales are increasing from May to October.
Also we can see several outliers.

In [ ]:
# Polar plots
mounthly_ts = ts_by_years.groupby('month').sum()
for i, game_code in enumerate(game_codes):
    fig, axes = plt.subplots(1, 2, figsize=(5,5), dpi= 80)
    plt.title(str(game_code))
    plt.polar(mounthly_ts.iloc[:, i])
    plt.show()

We can observe a high peak in December and slight tendency to increased sales in cold months

##### Trend, Season Decomposition

In [ ]:
rand5ts =  df_bfill.sample(5, axis=1, random_state=1)
for i, col in enumerate(rand5ts.columns):
    # Multiplicative Decomposition 
    #result_mul = seasonal_decompose(ts_rand_0.iloc[:, i], model='multiplicative', extrapolate_trend='freq')

    # Additive Decomposition
    result_add = seasonal_decompose(rand5ts.iloc[:, i], model='additive', extrapolate_trend='freq')

    # Plot
    plt.rcParams.update({'figure.figsize': (8,8)})
    #result_mul.plot().suptitle('Multiplicative Decompose' + str(col), fontsize=22)
    result_add.plot()

## Examinate on outliers

In [ ]:
# Let's explore on outliers common_sales

# import sesd

# outliers_indices = sesd.seasonal_esd(common_sales, hybrid=True, max_anomalies=5)
# for idx in outliers_indices:
#     print(f'Anomaly index: {idx}, anomaly value: {common_sales[idx]}')
# print('--------\n')

# # And compare it with single TS
# for i, col in enumerate(rand5ts.columns):
#     anomaly_index = sesd.seasonal_esd(rand5ts.iloc[:, i], hybrid=True, max_anomalies=5)
#     for idx in anomaly_index:
#         print(f'Anomaly index: {idx}, anomaly value: {rand5ts.iloc[idx, i]}')
#     print('\n')

We found only 4 outliers in time series of all sales. Found indexes are also occures in particular time series.

In [ ]:
# Smoothing data

rolled_knn = pd.DataFrame() 
for i, col in enumerate(df_knn.columns):
    rolled_knn.insert(i, col, df_knn.iloc[:, i].rolling(window=4).mean(), True)

fig, axes = plt.subplots(1, 2, figsize=(14,5))
rolled_knn[random2ts.columns[0]].plot(subplots=True, title='Rolling mean trend', ax=axes[0], label='Rolling mean trend', color='green')
df_knn[random2ts.columns[0]].plot(subplots=True, title='', ax=axes[0], label='Rolling mean trend', color='blue')
axes[0].legend(["Rolling mean trend", "Actual Data"])
rolled_knn[random2ts.columns[1]].plot(subplots=True, title='Rolling mean trend', ax=axes[1], label='Rolling mean trend', color='green')
df_knn[random2ts.columns[1]].plot(subplots=True, title='Rolling mean trend', ax=axes[1], label='Rolling mean trend', color='blue')
axes[1].legend(["Rolling mean trend", "Actual Data"])

## Stationarity Test

In [ ]:
# ADF test
def adf_nonstationarity_counter(df):
    nonstationarity_count = 0
    for i, col in enumerate(df.columns):
        test = adfuller(df.iloc[:, i])
        if test[0]> test[4]['5%']: 
            nonstationarity_count +=1
    return('Nonstationary ts number: ' + str(nonstationarity_count) + '\nShare of all TS: '+ str(nonstationarity_count/df.shape[1]))

In [ ]:
# initial state
print(adf_nonstationarity_counter(df_knn))

68 of 350 time series (19,4%) are non-stationary

### Using statmodels: Subtracting the Trend Component

In [ ]:
detrended = pd.DataFrame() 
for i, col in enumerate(df_knn.columns):
    # Additive Decomposition
    result_add = seasonal_decompose(df_knn.iloc[:, i], model='additive', extrapolate_trend='freq')
    detrended.insert(i, col, (df_knn.iloc[:, i].values - result_add.trend)/ result_add.seasonal, True)
    
print(adf_nonstationarity_counter(detrended))

### Differencing Series

In [ ]:
ts1diff = df_zeros.diff(periods=1).dropna()
print(adf_nonstationarity_counter(ts1diff))

Differencing TS shows better results. Only 1.1% of TS appears to be nonstationary

In [ ]:
ts1diff_knn = df_knn.diff(periods=1).dropna()
print(adf_nonstationarity_counter(ts1diff_knn))

In [ ]:
ts_rand_1diff = ts1diff.sample(4, axis=1)
ts_rand_1diff.plot(subplots=True, layout=(2,2), figsize=(8,8))
plt.show()

In [ ]:
ts2diff = df_knn.diff(periods=2).dropna()
print(adf_nonstationarity_counter(ts2diff))

Differencing 2 times is worse

## Autocorrelation

In [ ]:
#  Lag Scatter Plots

rand4ts = df_knn.sample(4, axis=1)
#c1, c2, c3, c4 = sns.color_palette("husl", 4)
ax1 = plt.subplot2grid((2,2), (0,0))
ax2 = plt.subplot2grid((2,2), (0,1))
ax3 = plt.subplot2grid((2,2), (1,0))
ax4 = plt.subplot2grid((2,2), (1,1))

lag_plot(df_knn[rand4ts.columns[0]], ax=ax1, alpha=0.5)
lag_plot(df_knn[rand4ts.columns[1]], ax=ax2, alpha=0.5)
lag_plot(df_knn[rand4ts.columns[2]], ax=ax3, alpha=0.5)
lag_plot(df_knn[rand4ts.columns[3]], ax=ax4, alpha=0.5)


Smoll (regular) values are correlated with their previous value. Large values are not, there are outliers

In [ ]:
# Autocorrelation plots

ax1 = plt.subplot2grid((2,2), (0,0))
ax2 = plt.subplot2grid((2,2), (0,1))
ax3 = plt.subplot2grid((2,2), (1,0))
ax4 = plt.subplot2grid((2,2), (1,1))

autocorrelation_plot(df_knn[rand4ts.columns[0]], ax=ax1)
autocorrelation_plot(df_knn[rand4ts.columns[1]], ax=ax2)
autocorrelation_plot(df_knn[rand4ts.columns[2]], ax=ax3)
autocorrelation_plot(df_knn[rand4ts.columns[3]], ax=ax4)


We can see autocorrelation with several first lags. And there is a peak between two holiday pre-New Year weeks.